In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 59.0 MB/s 
     |████████████████████████████████| 9.3 MB 19.9 MB/s 
     |████████████████████████████████| 108 kB 60.3 MB/s 


In [2]:
import pandas as pd
import numpy as np
from pennylane import numpy as np
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler

import pennylane as qml
from pennylane.templates.embeddings import AngleEmbedding, AmplitudeEmbedding
from pennylane.optimize import AdamOptimizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


import time
start = time.time()

In [3]:
df = pd.read_csv('/content/ClaMP_Integrated-5184.csv', sep=',')

In [4]:
df.drop(['packer_type'],axis=1,inplace=True)

In [5]:
from sklearn.utils import shuffle
df = shuffle(df)

In [6]:
# Separate class
class_0 = df[df['class'] == 0]
class_1 = df[df['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2488, 69)
class 1: (2722, 69)


In [7]:
df = df.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')

In [8]:
df.drop(['index'],axis=1,inplace=True)

In [9]:
df

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,296,4,1,0,1,...,1,1,3,0,6.112172,4.706247,553304,6.313146,1,0
1,144,3,4,65535,184,248,4,1,1,1,...,1,1,3,0,3.535598,0.000000,105984,5.623358,0,1
2,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,1,0.686928,0.003282,29400,2.845007,1,0
3,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.024795,3.082608,40448,5.998380,1,0
4,144,3,4,65535,184,280,4,1,0,1,...,1,1,3,0,6.042463,4.615791,178488,6.211368,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5205,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.753154,5.096242,745472,6.750296,0,0
5206,144,3,4,65535,184,128,3,1,0,1,...,1,1,2,1,6.776335,0.000000,402488,6.961778,0,0
5207,144,3,4,65535,184,248,4,1,0,1,...,1,1,3,0,6.191122,4.662779,61968,5.991154,0,0
5208,144,3,4,65535,184,128,4,1,0,1,...,1,1,3,0,7.479645,7.997533,180736,7.724639,0,1


In [10]:
df_5percent=df[0:4168]

In [11]:
df_5percent

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
0,144,3,4,65535,184,296,4,1,0,1,...,1,1,3,0,6.112172,4.706247,553304,6.313146,1,0
1,144,3,4,65535,184,248,4,1,1,1,...,1,1,3,0,3.535598,0.000000,105984,5.623358,0,1
2,144,3,4,65535,184,232,4,1,1,1,...,1,0,4,1,0.686928,0.003282,29400,2.845007,1,0
3,144,3,4,65535,184,232,5,1,0,1,...,1,1,4,0,6.024795,3.082608,40448,5.998380,1,0
4,144,3,4,65535,184,280,4,1,0,1,...,1,1,3,0,6.042463,4.615791,178488,6.211368,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4163,80,2,4,65535,184,512,7,0,0,1,...,1,1,6,0,6.503819,3.216717,107008,5.976842,0,0
4164,144,3,4,65535,184,128,5,1,0,1,...,1,2,3,0,6.480159,3.079727,3953640,7.987170,1,0
4165,144,3,4,65535,184,240,4,1,1,1,...,1,0,4,0,7.550765,7.784456,129536,7.338823,1,1
4166,144,3,4,65535,184,240,5,1,0,1,...,1,1,4,0,6.348522,6.954250,235520,6.447679,1,0


In [12]:
# Separate class
class_0 = df_5percent[df_5percent['class'] == 0]
class_1 = df_5percent[df_5percent['class'] == 1]# print the shape of the class
print('class 0:', class_0.shape)
print('class 1:', class_1.shape)

class 0: (2024, 69)
class 1: (2144, 69)


In [13]:
# Read out CSV and sets/samples creation

df_5percent = df_5percent.astype(float)
train,test = train_test_split(df_5percent, test_size=0.30, random_state=2)
train_set = train
test_set = test
train_set = train_set.sample(160)
test_set = test_set.sample(40)
np.random.seed(42)

In [14]:
df_5percent.info

<bound method DataFrame.info of       e_cblp  e_cp  e_cparhdr  e_maxalloc   e_sp  e_lfanew  NumberOfSections  \
0      144.0   3.0        4.0     65535.0  184.0     296.0               4.0   
1      144.0   3.0        4.0     65535.0  184.0     248.0               4.0   
2      144.0   3.0        4.0     65535.0  184.0     232.0               4.0   
3      144.0   3.0        4.0     65535.0  184.0     232.0               5.0   
4      144.0   3.0        4.0     65535.0  184.0     280.0               4.0   
...      ...   ...        ...         ...    ...       ...               ...   
4163    80.0   2.0        4.0     65535.0  184.0     512.0               7.0   
4164   144.0   3.0        4.0     65535.0  184.0     128.0               5.0   
4165   144.0   3.0        4.0     65535.0  184.0     240.0               4.0   
4166   144.0   3.0        4.0     65535.0  184.0     240.0               5.0   
4167   144.0   3.0        4.0     65535.0  184.0     232.0               5.0   

      C

In [15]:
df_5percent.describe()

,e_cblp,e_cp,e_cparhdr,e_maxalloc,e_sp,e_lfanew,NumberOfSections,CreationYear,FH_char0,FH_char1,...,LoaderFlags,sus_sections,non_sus_sections,packer,E_text,E_data,filesize,E_file,fileinfo,class
count,4168.000000,4168.000000,4168.000000,4168.000000,4168.000000,4168.000000,4168.000000,4168.000000,4168.000000,4168.0,...,4168.000000,4168.000000,4168.000000,4168.000000,4168.000000,4168.000000,4.168000e+03,4168.000000,4168.000000,4168.000000
mean,147.981766,2.949616,3.975048,65186.134117,199.125240,222.601727,4.634357,0.989203,0.363484,1.0,...,0.998800,1.328935,3.305422,0.153551,4.952217,2.536745,8.103459e+05,6.351018,0.545106,0.514395
std,571.381003,2.212788,0.311913,4704.908289,1012.397874,47.728602,1.843190,0.103356,0.481060,0.0,...,0.034619,1.537940,1.145959,0.360561,2.506516,2.678167,5.535648e+06,1.126346,0.498021,0.499853
min,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,1.000000,0.000000,0.000000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.536000e+03,0.939626,0.000000,0.000000
25%,144.000000,3.000000,4.000000,65535.000000,184.000000,208.000000,4.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,3.000000,0.000000,4.016450,0.000000,5.984200e+04,5.681256,0.000000,0.000000
50%,144.000000,3.000000,4.000000,65535.000000,184.000000,232.000000,5.000000,1.000000,0.000000,1.0,...,1.000000,1.000000,4.000000,0.000000,6.146129,1.600756,1.218560e+05,6.379328,1.000000,1.000000
75%,144.000000,3.000000,4.000000,65535.000000,184.000000,248.000000,5.000000,1.000000,1.000000,1.0,...,1.000000,2.000000,4.000000,0.000000,6.505517,4.685206,3.052840e+05,7.284223,1.000000,1.000000
max,37008.000000,144.000000,4.000000,65535.000000,65534.000000,592.000000,34.000000,1.000000,1.000000,1.0,...,1.000000,31.000000,8.000000,1.000000,7.999131,7.999620,1.657081e+08,7.999997,1.000000,1.000000


In [16]:
# Separation of labels

x_train = train_set
y_train = train_set[['class']]

x_test = test_set
y_test = test_set[['class']]

In [17]:
# Reduce dimensions using PCA to fit the dimensions with the qubits

n_dim = 2
pca = PCA(n_components=n_dim)
pca.fit(x_train)

x_train = pca.transform(x_train)

pca.fit(x_test)
x_test = pca.transform(x_test)

In [18]:
# Normalize

std_scale = StandardScaler().fit(x_train)
data = std_scale.transform(x_train)

std_scale = StandardScaler().fit(x_test)
x_test = std_scale.transform(x_test)

In [19]:
# Review the balance of the target variable in train

y_train.value_counts(normalize=True)*100

class
1.0      54.375
0.0      45.625
dtype: float64

In [20]:
# Review the balance of the target variable in test

y_test.value_counts(normalize=True)*100

class
1.0      55.0
0.0      45.0
dtype: float64

In [21]:
# Angle Encoding

num_qubits = n_dim

dev = qml.device('default.qubit', wires = num_qubits)

@qml.qnode(dev)
def circuit(parameters, data):
    for i in range(num_qubits):
        qml.Hadamard(wires = i)
    
    AngleEmbedding(features = data, wires = range(num_qubits), rotation = 'Y')
    
    qml.StronglyEntanglingLayers(weights = parameters, wires = range(num_qubits))
    
    return qml.expval(qml.PauliZ(0))

In [22]:
num_layers = 5
weights_init = 0.01 * np.random.randn(num_layers, num_qubits, 3, requires_grad=True)
bias_init = np.array(0.0, requires_grad=True)

print(weights_init, bias_init)

[[[ 0.00496714 -0.00138264  0.00647689]
  [ 0.0152303  -0.00234153 -0.00234137]]

 [[ 0.01579213  0.00767435 -0.00469474]
  [ 0.0054256  -0.00463418 -0.0046573 ]]

 [[ 0.00241962 -0.0191328  -0.01724918]
  [-0.00562288 -0.01012831  0.00314247]]

 [[-0.00908024 -0.01412304  0.01465649]
  [-0.00225776  0.00067528 -0.01424748]]

 [[-0.00544383  0.00110923 -0.01150994]
  [ 0.00375698 -0.00600639 -0.00291694]]] 0.0


In [23]:
circuit(weights_init, data[0])

tensor(0.12427427, requires_grad=True)

In [24]:
def variational_classifier(weights, bias, x):
    return circuit(weights, x) + bias

In [25]:
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

In [26]:
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss

In [27]:
def cost(weights, bias, X, Y):
    predictions = [variational_classifier(weights, bias, x) for x in X]
    return square_loss(Y, predictions)

In [28]:
Y = np.array(y_train.values[:,0] * 2 - np.ones(len(y_train.values[:,0])), requires_grad = False)  # shift label from {0, 1} to {-1, 1}
X = np.array(data, requires_grad=False)

for i in range(5):
    print("X = {}, Y = {: d}".format(list(X[i]), int(Y[i])))

X = [tensor(-0.33017194, requires_grad=False), tensor(-0.28572424, requires_grad=False)], Y = -1
X = [tensor(-0.23301382, requires_grad=False), tensor(-0.18492405, requires_grad=False)], Y =  1
X = [tensor(-0.23004963, requires_grad=False), tensor(-0.27472612, requires_grad=False)], Y = -1
X = [tensor(-0.23030505, requires_grad=False), tensor(-0.26815376, requires_grad=False)], Y = -1
X = [tensor(-0.2311851, requires_grad=False), tensor(-0.23853292, requires_grad=False)], Y =  1


In [29]:
opt = AdamOptimizer(stepsize=0.1, beta1=0.9, beta2=0.99, eps=1e-08)
batch_size = 10

In [30]:
%%time
weights = weights_init
bias = bias_init

wbest = 0
bbest = 0
abest = 0

for it in range(100):

    # weights update by one optimizer step

    batch_index = np.random.randint(0, len(X), (batch_size,))
    X_batch = X[batch_index]
    Y_batch = Y[batch_index]
    weights, bias, _, _ = opt.step(cost, weights, bias, X_batch, Y_batch)

    # Compute the accuracy
    predictions = [np.sign(variational_classifier(weights, bias, x)) for x in X]
    
    if accuracy(Y, predictions) > abest:
        wbest = weights
        bbest = bias
        abest = accuracy(Y, predictions)
        print('New best')

    acc = accuracy(Y, predictions)

    print(
        "Iter: {:5d} | Cost: {:0.7f} | Accuracy: {:0.7f} ".format(
            it + 1, cost(weights, bias, X, Y), acc
        )
    )

New best
Iter:     1 | Cost: 1.1266338 | Accuracy: 0.4687500 
Iter:     2 | Cost: 1.0841852 | Accuracy: 0.4687500 
New best
Iter:     3 | Cost: 0.9781241 | Accuracy: 0.5937500 
Iter:     4 | Cost: 1.0620725 | Accuracy: 0.5687500 
Iter:     5 | Cost: 1.1182447 | Accuracy: 0.5687500 
Iter:     6 | Cost: 0.9602850 | Accuracy: 0.5750000 
Iter:     7 | Cost: 1.0472892 | Accuracy: 0.4937500 
Iter:     8 | Cost: 0.9946097 | Accuracy: 0.4937500 
New best
Iter:     9 | Cost: 0.9407436 | Accuracy: 0.6000000 
Iter:    10 | Cost: 1.0069214 | Accuracy: 0.5812500 
Iter:    11 | Cost: 1.0772897 | Accuracy: 0.5812500 
Iter:    12 | Cost: 1.0730271 | Accuracy: 0.5812500 
Iter:    13 | Cost: 1.0340188 | Accuracy: 0.5812500 
Iter:    14 | Cost: 0.9528456 | Accuracy: 0.5812500 
Iter:    15 | Cost: 0.9551100 | Accuracy: 0.4500000 
Iter:    16 | Cost: 1.0133148 | Accuracy: 0.5000000 
Iter:    17 | Cost: 1.0800254 | Accuracy: 0.5000000 
Iter:    18 | Cost: 1.1432890 | Accuracy: 0.5000000 
Iter:    19 | Cost:

In [31]:
Yte = np.array(y_test.values[:,0] * 2 - np.ones(len(y_test.values[:,0])), requires_grad = False)
Xte = np.array(normalize(x_test), requires_grad=False)

In [32]:
predictions = [np.sign(variational_classifier(wbest, bbest, x)) for x in Xte]
pred = [np.sign(variational_classifier(wbest, bbest, x)) for x in X]
acc = accuracy(Yte, predictions)

print(f'Cost: {cost(wbest, bbest, Xte, Yte)}, Accuracy: {np.round(acc, 2) * 100}%')

Cost: 1.1118269240609564, Accuracy: 52.0%


In [33]:
pd.DataFrame((predictions, Yte), ('Predictions', 'Test')).T

,Predictions,Test
0,1.0,1.0
1,1.0,-1.0
2,1.0,-1.0
3,1.0,1.0
4,1.0,-1.0
5,1.0,1.0
6,-1.0,1.0
7,1.0,1.0
8,1.0,1.0
9,1.0,1.0


In [34]:
# Print the classification report and important metrics

print(metrics.classification_report(predictions,Yte))
print(metrics.precision_score(predictions,Yte))
print(metrics.recall_score(predictions,Yte))
print(metrics.f1_score(predictions,Yte))
print(metrics.balanced_accuracy_score(predictions,Yte))

              precision    recall  f1-score   support

        -1.0       0.06      0.33      0.10         3
         1.0       0.91      0.54      0.68        37

    accuracy                           0.53        40
   macro avg       0.48      0.44      0.39        40
weighted avg       0.85      0.53      0.63        40

0.9090909090909091
0.5405405405405406
0.6779661016949153
0.4369369369369369
